In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, pandas_udf, split
from pyspark.sql import SparkSession

topic = 'coin'
kafka_bootstrap_servers = 'kafka-div:9092'


spark = SparkSession.builder.appName("kafka-ex")\
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [6]:
from kafka import KafkaProducer
from pyspark.sql.functions import col, to_json
from pyspark.sql.types import StringType
import json

producer = KafkaProducer(bootstrap_servers=[kafka_bootstrap_servers])


In [26]:
data = [("2024-03-29 12:00:00", 0), ("2024-03-29 13:00:00", 1)]
df = spark.createDataFrame(data, ["timestamp", "check"])
df = df.withColumn("timestamp", col("timestamp").cast(StringType()))


In [27]:
df.show()

+-------------------+-----+
|          timestamp|check|
+-------------------+-----+
|2024-03-29 12:00:00|    0|
|2024-03-29 13:00:00|    1|
+-------------------+-----+



In [41]:
json_strings = df.toJSON().collect()

for row in json_strings:
    print(row)
    producer.send(topic, row.encode('utf-8'))


['{"timestamp":"2024-03-29 12:00:00","check":0}',
 '{"timestamp":"2024-03-29 13:00:00","check":1}']